# Introduction

Score just the bams

In [1]:
from Bio import SeqIO
from matplotlib import pyplot
import numpy
from pathlib import Path
import pandas
import pysam
from tqdm import tqdm
from xopen import xopen
from collections import Counter

In [2]:
experiments = pandas.read_csv("cached_experiments.tsv", sep="\t")

In [3]:
experiments

,experiment,description,simple_biosample_summary,file,output_type,file_size,bio_rep,tech_rep,target
0,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF318SKH,reads,255510907,1,1_1,cache/ENCSR293MOX_1_1_ENCFF318SKH_reads.fastq.gz
1,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF430RUG,unfiltered alignments,362638099,1,1_1,cache/ENCSR293MOX_1_1_ENCFF430RUG_unfiltered a...
2,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF211SQY,reads,387217252,1,1_1,cache/ENCSR081NRO_1_1_ENCFF211SQY_reads.fastq.gz
3,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF967OHL,unfiltered alignments,521377683,1,1_1,cache/ENCSR081NRO_1_1_ENCFF967OHL_unfiltered a...
4,ENCSR257JBF,excitatory cortical neurons,male adult (53 years),ENCFF919JFJ,reads,345607718,1,1_1,cache/ENCSR257JBF_1_1_ENCFF919JFJ_reads.fastq.gz
...,...,...,...,...,...,...,...,...,...
271,ENCSR462COR,RUSH brain E3348003,female adult (90 or above years) with Alzheime...,ENCFF565SOM,unfiltered alignments,1674565340,1,1_1,cache/ENCSR462COR_1_1_ENCFF565SOM_unfiltered a...
272,ENCSR257YUB,RUSH brain E4368365,male adult (90 or above years) with Alzheimer'...,ENCFF311CZO,reads,1533309113,1,1_1,cache/ENCSR257YUB_1_1_ENCFF311CZO_reads.fastq.gz
273,ENCSR257YUB,RUSH brain E4368365,male adult (90 or above years) with Alzheimer'...,ENCFF646VGI,unfiltered alignments,1858836988,1,1_1,cache/ENCSR257YUB_1_1_ENCFF646VGI_unfiltered a...
274,ENCSR690QHM,RUSH brain E6341028,female adult (90 or above years) with Alzheime...,ENCFF785KVJ,reads,1160071924,1,1_1,cache/ENCSR690QHM_1_1_ENCFF785KVJ_reads.fastq.gz


In [4]:
experiments[experiments["file"] == "ENCFF140UJC"]

,experiment,description,simple_biosample_summary,file,output_type,file_size,bio_rep,tech_rep,target


In [5]:
target = experiments.iloc[1]["target"]
target

'cache/ENCSR293MOX_1_1_ENCFF430RUG_unfiltered alignments.bam'

In [6]:
def score_bam_reads(filename, verbose=False):
    with pysam.AlignmentFile(filename, "rb") as inbam:
        read_names = Counter()
        read_flags = Counter()
        reads = {
            "primary": [],
            "non_primary": [],
            "unaligned": [],
            "total": [],
        }

        for read in inbam:
            read_names[read.query_name] += 1
            read_flags[read.flag] += 1
            mode = None
            if read.is_unmapped:
                mode = "unaligned"
            elif not read.is_unmapped and not (read.is_secondary or read.is_supplementary):
                mode = "primary"
            elif not read.is_unmapped and (read.is_secondary or read.is_supplementary):
                mode = "non_primary"
            else:
                print("We shouldn't reach this:", read)
                
            reads[mode].append(read.query_length)
            reads["total"].append(read.query_length)

        duplicates = 0
        for name in read_names:
            if read_names[name] > 1:
                duplicates += 1
        if verbose:
            print("duplicate", duplicates)
            for f in sorted(read_flags):
                print("0x{:04x} {}".format(f, read_flags[f]))
        summary = {}
        for key in reads:
            reads[key] = pandas.Series(reads[key])
            summary[key] = reads[key].describe()
            summary[key]["median"] = numpy.median(reads[key])
            
        return pandas.DataFrame(summary).unstack()

score_bam_reads(target, verbose=True)

duplicate 43882
0x0000 378354
0x0004 7030
0x0010 304786
0x0800 22630
0x0810 21923


primary      count     683140.000000
             mean        1209.884494
             std          866.069058
             min          300.000000
             25%          587.000000
             50%          913.000000
             75%         1583.000000
             max        20450.000000
             median       913.000000
non_primary  count      44553.000000
             mean        1268.912554
             std          920.516679
             min           81.000000
             25%          529.000000
             50%         1097.000000
             75%         1780.000000
             max        13765.000000
             median      1097.000000
unaligned    count       7030.000000
             mean         788.329587
             std          950.691038
             min          300.000000
             25%          383.000000
             50%          533.000000
             75%          815.000000
             max        14935.000000
             median       533.000000
t

In [7]:
reads = {}
for i, row in tqdm(experiments.iterrows(), total=experiments.shape[0]):
    name = "{}_{}".format(row["experiment"], row["tech_rep"])
    if row.output_type == "unfiltered alignments":
        reads[name] = score_bam_reads(row["target"])

reads = pandas.DataFrame(reads)

100%|██████████| 276/276 [37:55<00:00,  8.24s/it]


In [8]:
print(reads.T.to_csv())

,primary,primary,primary,primary,primary,primary,primary,primary,primary,non_primary,non_primary,non_primary,non_primary,non_primary,non_primary,non_primary,non_primary,non_primary,unaligned,unaligned,unaligned,unaligned,unaligned,unaligned,unaligned,unaligned,unaligned,total,total,total,total,total,total,total,total,total
,count,mean,std,min,25%,50%,75%,max,median,count,mean,std,min,25%,50%,75%,max,median,count,mean,std,min,25%,50%,75%,max,median,count,mean,std,min,25%,50%,75%,max,median
ENCSR293MOX_1_1,683140.0,1209.8844936616213,866.0690576381123,300.0,587.0,913.0,1583.0,20450.0,913.0,44553.0,1268.9125535878616,920.5166788471794,81.0,529.0,1097.0,1780.0,13765.0,1097.0,7030.0,788.329587482219,950.6910376385227,300.0,383.0,533.0,815.0,14935.0,533.0,734723.0,1209.430371718321,871.4094182225741,81.0,574.0,914.0,1591.0,20450.0,914.0
ENCSR081NRO_1_1,854940.0,1386.8435632909911,933.383142828224,300.0,730.0,1243.0,1743.0,20435.0,1243.0,61457.0,1341.8791512765022,963.0289312342494,80.0,598.0

In [9]:
reads.T.to_csv("bam_read_summary.csv")

In [10]:
reads.T

primary                                                   \
                     count         mean          std    min     25%     50%   
ENCSR293MOX_1_1   683140.0  1209.884494   866.069058  300.0   587.0   913.0   
ENCSR081NRO_1_1   854940.0  1386.843563   933.383143  300.0   730.0  1243.0   
ENCSR257JBF_1_2   770389.0  1540.944205   987.192926  300.0   855.0  1433.0   
ENCSR257JBF_1_1   715098.0  1471.351878   927.762674  300.0   825.0  1406.0   
ENCSR838WFC_1_1  2135001.0  1607.560022   717.466194   80.0  1045.0  1568.0   
...                    ...          ...          ...    ...     ...     ...   
ENCSR463IDK_1_1  2223209.0  1613.871873   998.851156   80.0   880.0  1431.0   
ENCSR316ZTD_1_1  2732454.0  1667.520060  1048.592255   80.0   915.0  1473.0   
ENCSR462COR_1_1  3014382.0  1381.958690   785.265512   80.0   859.0  1170.0   
ENCSR257YUB_1_1  2860061.0  1677.632293  1028.293523   80.0   931.0  1500.0   
ENCSR690QHM_1_1  2411154.0  1496.818379   952.845968   80.0   802.0  1283.0   

                                         non_primary  ... unaligned  \
                    75%      max  median       count  ...    median   
ENCSR293MOX_1_1  1583.0  20450.0   913.0     44553.0  ...     533.0   
ENCSR081NRO_1_1  1743.0  20435.0  1243.0     61457.0  ...     576.0   
ENCSR257JBF_1_2  1856.0  20390.0  1433.0     53707.0  ...    1337.0   
ENCSR257JBF_1_1  1774.0  19854.0  1406.0     50903.0  ...     992.0   
ENCSR838WFC_1_1  2082.0  12755.0  1568.0     13932.0  ...      72.0   
...                 ...      ...     ...         ...  ...       ...   
ENCSR463IDK_1_1  2134.0  12672.0  1431.0     77819.0  ...     888.0   
ENCSR316ZTD_1_1  2214.0  14483.0  1473.0     95396.0  ...    1162.0   
ENCSR462COR_1_1  1801.0  12239.0  1170.0     60717.0  ...    1301.0   
ENCSR257YUB_1_1  2202.0  15034.0  1500.0    101790.0  ...    1084.0   
ENCSR690QHM_1_1  1994.0  12805.0  1283.0     85740.0  ...     899.0   

                     total                                                  \
                     count         mean          std   min     25%     50%   
ENCSR293MOX_1_1   734723.0  1209.430372   871.409418  81.0   574.0   914.0   
ENCSR081NRO_1_1   918423.0  1383.144401   936.996919  80.0   722.0  1235.0   
ENCSR257JBF_1_2   829052.0  1541.419548   991.463472  80.0   847.0  1432.0   
ENCSR257JBF_1_1   773020.0  1466.718856   930.325307  80.0   814.0  1402.0   
ENCSR838WFC_1_1  2151100.0  1599.280992   722.153420  50.0  1035.0  1559.0   
...                    ...          ...          ...   ...     ...     ...   
ENCSR463IDK_1_1  2333310.0  1566.893590  1017.186033  50.0   817.0  1383.0   
ENCSR316ZTD_1_1  2891879.0  1621.845931  1077.331815  50.0   838.0  1423.0   
ENCSR462COR_1_1  3213058.0  1371.770689   832.975195  50.0   814.0  1150.0   
ENCSR257YUB_1_1  3006233.0  1628.964647  1052.666125  50.0   869.0  1449.0   
ENCSR690QHM_1_1  2557488.0  1452.600399   973.353283  50.0   732.0  1232.0   

                                          
                    75%      max  median  
ENCSR293MOX_1_1  1591.0  20450.0   914.0  
ENCSR081NRO_1_1  1743.0  20435.0  1235.0  
ENCSR257JBF_1_2  1870.0  20390.0  1432.0  
ENCSR257JBF_1_1  1780.0  19854.0  1402.0  
ENCSR838WFC_1_1  2077.0  12755.0  1559.0  
...                 ...      ...     ...  
ENCSR463IDK_1_1  2097.0  12672.0  1383.0  
ENCSR316ZTD_1_1  2178.0  14483.0  1423.0  
ENCSR462COR_1_1  1802.0  12239.0  1150.0  
ENCSR257YUB_1_1  2167.0  15034.0  1449.0  
ENCSR690QHM_1_1  1966.0  12805.0  1232.0  

[138 rows x 36 columns]